In [1]:
import functools

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [5]:
# 让 numpy 数据更易读。
np.set_printoptions(precision=3, suppress=True)

In [10]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

dataset = tf.data.experimental.make_csv_dataset(
     ...,
     column_names=CSV_COLUMNS,
     ...)


SyntaxError: positional argument follows keyword argument (<ipython-input-10-24b3847a11bc>, line 6)

In [11]:
dataset = tf.data.experimental.make_csv_dataset(
  ...,
  select_columns = columns_to_use, 
  ...)


SyntaxError: positional argument follows keyword argument (<ipython-input-11-6bb035305282>, line 4)

In [12]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [13]:
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # 为了示例更容易展示，手动设置较小的值
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [14]:
examples, labels = next(iter(raw_train_data)) # 第一个批次
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'male', b'male', b'male', b'male', b'female', b'male', b'female',
       b'male', b'female', b'male', b'male', b'male'], dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([36., 23., 40., 26., 39., 71., 31., 28., 28., 16., 41., 34.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 5, 0, 0, 0, 2, 0, 0, 0])>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([12.875, 10.5  ,  0.   ,  7.896, 31.275, 49.504,  8.683,  7.896,
       22.358,  9.5  ,  7.125, 13.   ], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'Second', b'Second', b'First', b'Third', b'Third', b'First',
       b'Third', b'Third', b'Third', b'Third', b'Third', b'Second'],
      dtype=object)>), ('deck', <t

In [15]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [16]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [17]:
def process_continuous_data(mean, data):
  # 标准化数据
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

In [18]:
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
  num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
  numerical_columns.append(num_col)

In [19]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numerical_columns)

In [20]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [21]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [22]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
53/53 [==============================] - 1s 27ms/step - loss: 0.5265 - accuracy: 0.7544
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4392 - accuracy: 0.8150
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4316 - accuracy: 0.8166
Epoch 4/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4112 - accuracy: 0.8166
Epoch 5/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4110 - accuracy: 0.8150
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3943 - accuracy: 0.8309
Epoch 7/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3875 - accuracy: 0.8293
Epoch 8/20
53/53 [==============================] - 0s 3ms/step -

In [23]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     22/Unknown - 0s 13ms/step - loss: 0.4510 - accuracy: 0.8144

Test Loss 0.4510360075668855, Test Accuracy 0.814393937587738


In [24]:
predictions = model.predict(test_data)

# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 83.31%  | Actual outcome:  SURVIVED
Predicted survival: 13.99%  | Actual outcome:  SURVIVED
Predicted survival: 24.24%  | Actual outcome:  DIED
Predicted survival: 9.65%  | Actual outcome:  DIED
Predicted survival: 99.52%  | Actual outcome:  DIED
Predicted survival: 8.31%  | Actual outcome:  DIED
Predicted survival: 99.54%  | Actual outcome:  SURVIVED
Predicted survival: 3.11%  | Actual outcome:  DIED
Predicted survival: 9.83%  | Actual outcome:  DIED
Predicted survival: 53.01%  | Actual outcome:  DIED
